In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt
import requests
from bs4 import BeautifulSoup
import tabula
import math

## Funções

In [4]:
def download_file(url,path):
    '''
    This function download the pdfs files from urls.
    
    Parameters:
        url: the url file to download.
        path: the path to save the pdf file after download.
        
    Return:
        Local pdf file path.
    '''
    local_filename = url.split('/')[-1]
    local_filename = path+'/'+local_filename 
    
    if(local_filename[-4:] != '.pdf'):
        local_filename = local_filename + '.pdf'
        
    r = requests.get(url)
    f = open(local_filename, 'wb')
    
    for chunk in r.iter_content(chunk_size=512 * 1024): 
        if chunk: 
            f.write(chunk)
    f.close()
    return local_filename

In [5]:
def create_folder(path):
    '''
    Function to create folder to save the files.
    
    Parameter:
        - path: absolute path file name.
    Return:
        - status of creation file.
    '''

    try:
        if(os.path.isdir(path)):
            print ("A pasta %s já existe" % path)
            return True
        os.mkdir(path)
    except OSError:
        print ("Criação da pasta %s falhou" % path)
        return False
    else:
        print ("A pasta %s foi criada com sucesso" % path)
        
    return True


results = 'resultados'
years = range(2016,2020)

if(create_folder(results)):
    for year in years:
        create_folder(results+'/'+str(year))

A pasta resultados já existe
A pasta resultados/2016 já existe
A pasta resultados/2017 já existe
A pasta resultados/2018 já existe
A pasta resultados/2019 já existe


In [3]:
def get_pdf_filename(dir='\\resultados'):
    '''
    Fuction to get absolute file name.
    
    Parameter:
        - dir: initial directory to reveal all the absolute filename.
    Return:
        - list(str): list of all the absolute filename.
    '''
    path = os.getcwd()+dir

    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.pdf' in file:
                files.append(os.path.join(r, file))

    return files

## Coletando e armazenando os dados da Olimpíada Nacional de Ciências (ONC)

### ONC 2019

In [8]:
url = "https://onciencias.org/resultado/resultado-final-onc-2019/"
url_home = 'https://onciencias.org/'

page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data)

links = []
# Encontro todos as tags 'a' e seleciono apenas as que possuem a substring 'medalha' ou 'mencao-honrosa'
for link in soup.find_all('a'):
    temp_link = link.get('href')
    if(str(temp_link).find('medalha') != -1 or str(temp_link).find('mencao-honrosa') != -1):
        links.append(url_home+temp_link[1:])
        #print(url_home+temp_link[1:])

# Extrair dos links as informações da série e da premiação para serem incluídas no dataframe
serie = []
medalhas = []
for link in links:
    temp_splited_link = link.split('/')
    serie.append(temp_splited_link[-2])
    medalhas.append(temp_splited_link[-1])


In [9]:
# Leitura dos dados da página e inclusão das informações no dataframe
for a,b,c in zip(serie,medalhas,links):
    df = pd.read_html(c)[0]
    df['serie'] = a
    df['medalha'] = b
    dfs.append(df)
appended_data = pd.concat(dfs)

In [10]:
appended_data.head()

,Nome do Aluno,Escola,Estado,Cidade,Unnamed: 4,serie,medalha
0,LARA DANTAS DE OLIVEIRA MOISES,7 DE SETEMBRO COLEGIO - NGS,CEARÁ,Fortaleza,NaN,9-ano-ensino-fundamental,medalha-ouro
1,NAILTON GAMA DE CASTRO A,MASTER COLEGIO,CEARÁ,Fortaleza,NaN,9-ano-ensino-fundamental,medalha-ouro
2,RAFAEL MORENO RIBEIRO,COLEGIO MILITAR DE SALVADOR,BAHIA,Salvador,NaN,9-ano-ensino-fundamental,medalha-ouro
3,ARTHUR PINTO LORENZO,COLEGIO MILITAR DE SALVADOR,BAHIA,Salvador,NaN,9-ano-ensino-fundamental,medalha-ouro
4,GUILHERME Z. HAUSSEN,COLEGIO ANCHIETA,RIO GRANDE DO SUL,Porto Alegre,NaN,9-ano-ensino-fundamental,medalha-ouro


In [42]:
# Separação em diferentes dataframes de acordo com a série do participante para salvar os dados em arquivos diferentes
series = list(appended_data['serie'].unique())
for serie in series:
    temp_df = appended_data[appended_data['serie'] == serie]
    temp_df.to_csv('resultados/2019/'+serie.replace('-','_')+'.csv')

### ONC 2018

In [11]:
# lista dos links para o download dos arquivos pdf
links_onc_2018= ['http://onciencias.org/resultados/download/15',
                'http://onciencias.org/resultados/download/16',
                'http://onciencias.org/resultados/download/17',
                'http://onciencias.org/resultados/download/18']

In [35]:
# Acessando as páginas de download dos arquivos do resultados da olimpíada e salvando os arquivos pdf
for i in links_onc_2018:
    download_file(i,'resultados/2018')

### ONC 2016

In [12]:
# lista dos links para o download dos arquivos pdf
links_onc_2016 = ['https://onciencias.org/resultados/download/9/ResultadoONC2016-Nivel%20D.pdf',
'https://onciencias.org/resultados/download/8/ResultadoONC2016-Nivel%20C.pdf',
'https://onciencias.org/resultados/download/7/ResultadoONC2016-Nivel%20B.pdf',
'https://onciencias.org/resultados/download/6/ResultadoONC2016-Nivel%20A.pdf']

In [36]:
# Acessando as páginas de download dos arquivos do resultados da olimpíada e salvando os arquivos pdf
for i in links_onc_2016:
    download_file(i,'resultados/2016')

## Processando os arquivos pdf

### Processando os arquivos pdf dos resultados de 2016

- Listagem dos arquivos de 2016

### First Try

In [4]:
files_2016 = get_pdf_filename('\\resultados\\2016')
files_2016

['C:\\Users\\pedro\\Projetos\\onc\\resultados\\2016\\ResultadoONC2016-Nivel%20A.pdf',
 'C:\\Users\\pedro\\Projetos\\onc\\resultados\\2016\\ResultadoONC2016-Nivel%20B.pdf',
 'C:\\Users\\pedro\\Projetos\\onc\\resultados\\2016\\ResultadoONC2016-Nivel%20C.pdf',
 'C:\\Users\\pedro\\Projetos\\onc\\resultados\\2016\\ResultadoONC2016-Nivel%20D.pdf']

- Conversão dos arquivos pdf em csv

In [5]:
for f in files_2016:
    tabula.convert_into(f, f.replace('.pdf','.csv'), output_format="csv", pages='all')

- Conferindo os arquivos

In [6]:
pd.read_csv(files_2016[0].replace('.pdf','.csv'),encoding='latin-1').tail()

,Nome,Escola,Cidade,UF,Nota,Escore
1452,Letícia Furtado Silva,E E Pe Anchieta,Coqueiral,MG,NaN,NaN
1453,NaN,NaN,NaN,NaN,NaN,NaN
1454,Luis Felipe Barbosa Lopes,Colégio Bom Jesus Santo Antonio,Rolandia,PR,NaN,NaN
1455,NaN,NaN,NaN,NaN,NaN,NaN
1456,Lucas Morais Ferreira,Ceefmti Daniel Comboni,Ecoporanga,ES,NaN,NaN


In [7]:
tdf = pd.read_csv(files_2016[0].replace('.pdf','.csv'),encoding='latin-1')

In [10]:
tdf.iloc[-1]['Nome'] == last_name_a

False

In [11]:
last_names = ['Tcharly Miguel Santana Santos',
'Juliana Barros Navarro',
'Gabriel Pita Rios',
'Matheus Fernandes Flores']

> Após ler o arquivo pdf, identifiquei que a última página do primeiro arquivo não foi lida. Isso pode ser corrigido alterando a área de seleção. [Link do tutorial](https://aegis4048.github.io/parse-pdf-files-while-retaining-structure-with-tabula-py).

In [10]:
def read_pdffile(pdf_filename):
    tables = tabula.read_pdf(pdf_filename, output_format="json", pages=2, silent=True)
    top = tables[0]["top"]
    left = tables[0]["left"]
    bottom = tables[0]["height"]+30
    right = tables[0]["width"]
    left = 0
    top = 0
    test_area = [top, left, bottom, right]
    print(test_area)
    return tabula.read_pdf(pdf_filename,multiple_tables=True,pages="all",area=test_area,silent=True)    

In [9]:
dfs = read_pdffile(files_2016[3])
dfs[-2].tail()

[22.210886, 0, 764.7825927734375, 524.549072265625]


,Emerson Juarez Ribeiro Martins Farias,Vale Do Curtume E E F M,Nova Russas,CE,Unnamed: 0
26,Gabriel Pradela,Colégio Objetivo Arujá,Aruja,SP,NaN
27,Gabriel Kashiwazaki,Colégio Elite,Campinas,SP,NaN
28,Renata Cristiane Rodrigues Ferreira,Colégio Objetivo Mairiporã,Mairipora,SP,NaN
29,Fernanda Lopes Ramalho,Etec De Monte Mor,Monte Mor,SP,NaN
30,Matheus Fernandes Flores,Votuporanguense De Ensino Escola,Votuporanga,SP,NaN


In [11]:
dfs[-3].tail()

,Cairo Henrique Cardoso Pereira,Sesi 268 Centro Educacional,Presidente Epitacio,SP,Unnamed: 0
43,Laíla De Moura Luz,Campus Picos Do Instituto Federal\rDo Piaui,Picos,PI,NaN
44,Vinícius Maia Neto,Colégio Nossa Senhora Do Amparo,Barra Mansa,RJ,NaN
45,Lucas Christyan Da Rocha Oliveira,Colégio Diocesano Santa Luzia,Mossoro,RN,NaN
46,Giorgi Bruno Soares De Sousa,IFRN - Cidade Alta,Natal,RN,NaN
47,Matheus Cavalcante Muricy,Centro Educacional Objetivo,Porto Velho,RO,NaN


In [99]:
pdf_filename = files_2016[3]
tables = tabula.read_pdf(pdf_filename, output_format="json", pages=9, silent=True)
top = tables[0]["top"]
left = tables[0]["left"]
bottom = tables[0]["height"]+30
right = tables[0]["width"]

test_area = [top, 0, bottom, right]

tdf1 = tabula.read_pdf(pdf_filename,multiple_tables=True,pages="9",area=test_area,silent=True)    

In [96]:
def read_page(pdf_filename,number_page):
    tables = tabula.read_pdf(pdf_filename, output_format="json", pages=number_page, silent=True)
    top = tables[0]["top"]
    left = tables[0]["left"]
    bottom = tables[0]["height"]+30
    right = tables[0]["width"]
    left = 0
    top = 0
    test_area = [top, left, bottom, right]
    #print(test_area)
    return tabula.read_pdf(pdf_filename,multiple_tables=False,pages=number_page,area=test_area,silent=True)    

In [193]:
cols = ['Nome', 'Escola', 'Cidade', 'UF', 'Nota']
pdf_filename = files_2016[3]

try:
    dfs = []
    for i in range(1,12):
        temp_df = read_page(pdf_filename,i)[0]
        temp_df.columns = cols # Update columns name
        dfs.append(temp_df)
except Exception as e:
    print(e)
    
dfs = pd.concat(dfs,ignore_index=True) # Concat the dataframes
dfs['Nota'] = dfs['Nota'].str.replace(',','.') # Replace , to .
dfs['Nota'] = pd.to_numeric(dfs['Nota'], downcast="float") # Convert string column to float

In [207]:
# Identify the inteval of the medalhists
medal_index = dfs.index[dfs['Nome'].isin(['O U R O','P R A T A','B R O N Z E','M E N Ç Ã O    H O N R O S A'])]
medal_index

Int64Index([1, 8, 18, 37], dtype='int64')

In [263]:
def test(row):
    
    if(medal_index[0] < row.name and row.name < medal_index[1]):
        return 'Ouro'
    elif(medal_index[1] < row.name and row.name < medal_index[2]):
        return 'Prata'
    elif(medal_index[2] < row.name and row.name < medal_index[3]):
        return 'Bronze'
    elif(medal_index[3] < row.name and math.isnan(row['Nota'])):
        return 'Honra'
    else:
        return 'Nenhuma'

In [270]:
# Setting medal column
dfs['medal'] = dfs.apply(test,axis=1)

In [269]:
dfs['medal'].head(40)

0     Nenhuma
1     Nenhuma
2        Ouro
3        Ouro
4        Ouro
5        Ouro
6        Ouro
7        Ouro
8     Nenhuma
9       Prata
10      Prata
11      Prata
12      Prata
13      Prata
14      Prata
15      Prata
16      Prata
17      Prata
18    Nenhuma
19     Bronze
20     Bronze
21     Bronze
22     Bronze
23     Bronze
24     Bronze
25     Bronze
26     Bronze
27     Bronze
28     Bronze
29     Bronze
30     Bronze
31     Bronze
32     Bronze
33     Bronze
34     Bronze
35     Bronze
36     Bronze
37    Nenhuma
38    Nenhuma
39    Nenhuma
Name: medal, dtype: object

In [262]:
dfs['Nota'] = pd.to_numeric(dfs['Nota'], downcast="float")

In [254]:
not math.isnan(dfs.iloc[-1]['Nota'])

False

In [252]:
dfs.iloc[100]['Nota']

'41,5'

- Salvando o arquivo pdf em formato csv

In [123]:
df[0].to_csv(files_2016[0].replace('.pdf','.csv'),index=False)

- Removendo linhas com valores nulos

In [10]:
pd.read_csv(files_2016[0].replace('.pdf','.csv'),encoding='utf8').tail()

,Nome,Escola,Cidade,UF,Nota,Escore
1370,Bruna Rafaela Barbosa E Silva,Dr Eloy de Souza,Acu,RN,NaN,NaN
1371,NaN,NaN,NaN,NaN,NaN,NaN
1372,Tanise Vieira,Colégio Dom Feliciano,Gravatai,RS,NaN,NaN
1373,NaN,NaN,NaN,NaN,NaN,NaN
1374,Tcharly Miguel Santana Santos,Escola Viva,Carmopolis,SE,NaN,NaN


In [34]:
df = pd.read_csv(files_2016[0].replace('.pdf','.csv'),encoding='utf8')
df.head()

,Nome,Escola,Cidade,UF,Nota,Escore
0,O U R O,NaN,NaN,NaN,NaN,NaN
1,Pedro Jatobá Arteiro,Colégio Santa Maria,Recife,PE,78,"100,0"
2,Amanda Onofre Neves Nóbrega,Antares Colégio,Fortaleza,CE,"77,5","99,4"
3,Thiago Eklésio Silveira Peixoto,Farias Brito,Fortaleza,CE,76,"97,4"
4,Marco Antônio de Lima Sampaio,Colégio Contato,Maceio,AL,75,"96,2"


In [15]:
df.dropna(how='all',inplace=True)
df.tail()

,Nome,Escola,Cidade,UF,Nota,Escore
1366,Everton Daniel da Silva,Escola Aluisio Germano,Carpina,PE,NaN,NaN
1368,Alexandre da Silva Gomes,Dr Eloy de Souza,Acu,RN,NaN,NaN
1370,Bruna Rafaela Barbosa E Silva,Dr Eloy de Souza,Acu,RN,NaN,NaN
1372,Tanise Vieira,Colégio Dom Feliciano,Gravatai,RS,NaN,NaN
1374,Tcharly Miguel Santana Santos,Escola Viva,Carmopolis,SE,NaN,NaN


In [35]:
df.sample(10)

,Nome,Escola,Cidade,UF,Nota,Escore
932,NaN,NaN,NaN,NaN,NaN,NaN
312,NaN,NaN,NaN,NaN,NaN,NaN
723,Iara Dorigon,Colégio Olimpo,Palmas,TO,NaN,NaN
695,Gabriel Rodrigues Carrijo,Col Exitus,Uberlandia,MG,NaN,NaN
536,Paulo Vitor Silva Santos,Colégio Cristo Rei,Maceio,AL,NaN,NaN
41,Joaquim Miguel Moreira Santiago,Ari de Sá Cavalcante,Fortaleza,CE,"67,5","86,54"
321,Ana Gabriela Ponte Farias,NaN,Antares Colégio,Fortaleza,CE,NaN
807,NaN,NaN,NaN,NaN,NaN,NaN
1241,Guilherme Gomes de Souza,EMEF Rodrigo de Argolo Caracas,Guaramiranga,CE,NaN,NaN
510,Ana Luiza Correia Veras,Antares Colégio,Fortaleza,CE,NaN,NaN


In [ ]:
125

In [61]:
def read_pdf(filename):
    tables = tabula.read_pdf(filename, output_format="json", pages=2, silent=True)
    top = tables[0]["top"]
    left = tables[0]["left"]
    bottom = tables[0]["height"] + top
    right = tables[0]["width"] + left

    test_area = [top, left, bottom, right]

    return tabula.read_pdf(filename,multiple_tables=True,pages="all",area=test_area,silent=True)

In [ ]:
df = read_pdf(files_2016[0])

In [67]:
df[2].shape

(50, 6)

In [70]:
df[3].head(20)

,Unnamed: 0,52,Unnamed: 1
0,NaN,52,NaN
1,NaN,52,NaN
2,NaN,"51,5",NaN
3,NaN,51,NaN
4,NaN,51,NaN
5,NaN,51,NaN
6,NaN,51,NaN
7,NaN,51,NaN
8,NaN,51,NaN
9,NaN,51,NaN


In [78]:
while True:
    try:
        df = tabula.read_pdf(files_2016[0],multiple_tables=True,pages=21,silent=True)
        break
    except Exception:
        print("Oops!  That was no valid number.  Try again...")
        break

Error from tabula-java:
Exception in thread "main" java.lang.IndexOutOfBoundsException: Page number does not exist
	at technology.tabula.ObjectExtractor.extractPage(ObjectExtractor.java:19)
	at technology.tabula.PageIterator.next(PageIterator.java:29)
	at technology.tabula.CommandLineApp.extractFile(CommandLineApp.java:165)
	at technology.tabula.CommandLineApp.extractFileTables(CommandLineApp.java:128)
	at technology.tabula.CommandLineApp.extractTables(CommandLineApp.java:109)
	at technology.tabula.CommandLineApp.main(CommandLineApp.java:79)




Oops!  That was no valid number.  Try again...
